# Development Workflow

## File-based Submission

This demonstrates the basic workflow for submitting a Flamingo job to Ray
from a configuration stored as a local file.

The job configuration is stored as a YAML file in a the local `configs` directory,
and that directory is specified as the working directory of the Ray runtime environment upon submission.

In [ ]:
# Required imports
import os
from pathlib import Path

from ray.job_submission import JobSubmissionClient

In [ ]:
# Create a submission client bound to a Ray cluster
# Note: You will likely have to update the cluster address shown below
client = JobSubmissionClient("http://10.146.174.91:8265")

In [ ]:
# Determine local module path for the flamingo repo
# In theory this workflow is possible without having the flamingo package installed locally,
# but this is a convenient means to access the local module path
import flamingo

flamingo_module = Path(flamingo.__file__).parent

In [ ]:
# Construct the runtime environment for your job submission
# py_modules contains the path to the local flamingo module directory
# pip contains an export of the dependencies for the flamingo package (see CONTRIBUTING.md for how to generate)
runtime_env = {
    "working_dir": "configs",
    "env_vars": {"WANDB_API_KEY": os.environ["WANDB_API_KEY"]},  # If running a job that uses W&B
    "py_modules": [str(flamingo_module)],
    "pip": "requirements.txt",  # See CONTRIBUTING.md for how to generate this
}

In [ ]:
# Submit the job to the Ray cluster
# Note: flamingo is invoked by 'python -m flamingo' since the CLI is not installed in the environment
client.submit_job(
    entrypoint=f"python -m flamingo run simple --config simple_config.yaml",
    runtime_env=runtime_env,
)

## Iterative Submission

It is also possible to submit Flamingo jobs using a fully Python/Jupyter driven workflow.

In this case, the Flamingo job configuration is instantiated in your Python script
and written to a temporary directory for submission. 

The Ray working directory is based off this temporary YAML file location.

This approach is convenient if you want to run sweeps over parameter ranges
and use a Python script/Jupyter notebook as your local "driver" for the workflow.

In [ ]:
# Required imports
import os
from pathlib import Path

from ray.job_submission import JobSubmissionClient

In [ ]:
# Create a submission client bound to a Ray cluster
# Note: You will likely have to update the cluster address shown below
client = JobSubmissionClient("http://10.147.154.77:8265")

In [ ]:
# Determine local module path for the flamingo repo
# In theory this workflow is possible without having the flamingo package installed locally,
# but this is a convenient means to access the local module path
import flamingo

flamingo_module = Path(flamingo.__file__).parent

In [ ]:
import os

from flamingo.jobs.simple import SimpleJobConfig

# Generate job configs programatically for sweeps over parameter ranges
magic_numbers = [0, 10, 20, 40]

for number in magic_numbers:
    # Instantitate config in your workflow script
    # You may also want to read a "base" config from file with some suitable defaults
    config = SimpleJobConfig(magic_number=number)

    # `config_path` is the fully qualified path to the config file on your local filesystem
    with config.to_tempfile(name="config.yaml") as config_path:
        # `config_path.parent` is the working directory
        runtime_env = {
            "working_dir": str(config_path.parent),
            "env_vars": {"WANDB_API_KEY": os.environ["WANDB_API_KEY"]},
            "py_modules": [str(flamingo_module)],
            "pip": "requirements.txt",  # See CONTRIBUTING.md for how to generate this
        }

        # `config_path.name` is the file name within the working directory, i.e., "config.yaml"
        client.submit_job(
            entrypoint=f"python -m flamingo run simple --config {config_path.name}",
            runtime_env=runtime_env,
        )